In [ ]:
import pathlib
from joblib import load

In [ ]:
# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open('sql/footy_tipping_data.sql', 'r') as file:
    query = file.read()

footy_tipping_data = pd.read_sql_query(query, con)

# Don't forget to close the connection
con.close()

footy_tipping_data

In [ ]:
# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Define the directory and filename where you want to save your model
model_dir = project_root / "models"

# Load the model and other objects
footy_tipper_model = load(model_dir / "footy_tipper_model.joblib")
label_encoder = load(model_dir / "label_encoder.joblib")

In [ ]:
predictions_df = mf.model_predictions(footy_tipper_model, X_inference, label_encoder, game_id_inference)
predictions_df

In [ ]:
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file and create table
with open('sql/create_table.sql', 'r') as file:
    create_table_query = file.read()
con.execute(create_table_query)

# Read SQL query from external SQL file for insertion
with open('sql/insert_into_table.sql', 'r') as file:
    insert_into_table_query = file.read()

# Write each row in the DataFrame to the database
for index, row in predictions_df.iterrows():
    con.execute(insert_into_table_query, (
        row['game_id'], 
        row['home_team_result'],
        row['home_team_win_prob'],
        row['home_team_lose_prob']
    ))

# Commit the transaction
con.commit()

# Close the connection
con.close()